## 用於訓練結果之評估，如果要評估比賽時本團隊訓練好的模型，請執行 team4762 evaluation.py

In [ ]:
import torch
from my_data import prepare_input, prepare_label, CustomDataset
from torch.utils.data import DataLoader
from my_model import ResidualBlock, ResViT, ResViT_kyu, ResViT_PS
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda:0" if USE_CUDA else "cpu")
print(device)

## 評估dan之驗證集平均準確率

In [ ]:
df = open('./29_Training Dataset/Training Dataset/dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df] #把前面和下棋無關的字串分隔開，並且只取後面有關的字串
pre_data_num = 0
bs = 256
train_games, val_games = train_test_split(games, test_size=0.001, random_state=64)
val_x_list = []
val_y_list = []
val_x_pre_eight_list = []
for val_game_num, val_game in enumerate(val_games):
    val_move_list = val_game.split(',')

    for idx, val_move in enumerate(val_move_list):
        val_x = prepare_input(val_move_list[:idx])
        val_x_list.append(val_x)
        val_y = prepare_label(val_move_list[idx])
        val_y = torch.nn.functional.one_hot(val_y, 19*19)
        val_y = torch.tensor(val_y, dtype=torch.float32)
        val_y_list.append(val_y)

        if((idx+1) <= pre_data_num):
            for i in range(idx):
                val_x = torch.cat((val_x, val_x_list[idx-1-i][:2]), dim=0)
            for i in range(pre_data_num - idx):
                val_x = torch.cat((val_x, torch.zeros((2, 19, 19))), dim=0)
               
        elif((idx+1) > pre_data_num):
            for i in range(pre_data_num):
                val_x = torch.cat((val_x, val_x_list[idx-1-i][:2]), dim=0)
            
        val_x_pre_eight_list.append(val_x)


val_x_pre_eight_stack = torch.stack(val_x_pre_eight_list)
val_y_stack = torch.stack(val_y_list)

val_x_pre_eight_stack = val_x_pre_eight_stack.to(device)
val_y_stack = val_y_stack.to(device)

val_dataset = CustomDataset(val_x_pre_eight_stack, val_y_stack)
data_loader_val = DataLoader(dataset=val_dataset, batch_size=bs, shuffle=False)

model = torch.load('./dan_models/model2000.pth')
model.eval()
accuracy_list_top1 = []
accuracy_list_top5 = []
with torch.no_grad():  # Disable gradient calculation during validation
    for val_x, val_y in tqdm(data_loader_val):
        predictions = model(val_x)
        top_k_values, top_k_indices = torch.topk(predictions, k=5, dim=1)
        val_y_values, val_y_indices = torch.topk(val_y, k=1)
        correct_predictions_top1 = torch.eq(top_k_indices[:, 0], val_y_indices.squeeze())
        correct_predictions_top5 = torch.any(torch.eq(top_k_indices, val_y_indices), dim=1)
    
        accuracy_top1 = torch.sum(correct_predictions_top1  == True) / len(val_y)
        accuracy_top5 = torch.sum(correct_predictions_top5  == True) / len(val_y)
        accuracy_list_top1.append(accuracy_top1.item())
        accuracy_list_top5.append(accuracy_top5.item())
        
        # print(f'Accuracy_top1: {accuracy_top1 * 100:.2f}%')
        # print(f'Accuracy_top5: {accuracy_top5 * 100:.2f}%')
print(f'top 1 avg acc:{sum(accuracy_list_top1)/len(accuracy_list_top1)}')
print(f'top 5 avg acc:{sum(accuracy_list_top5)/len(accuracy_list_top5)}')

## 評估kyu之驗證集平均準確率

In [ ]:
df = open('./29_Training Dataset/Training Dataset/kyu_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df] #把前面和下棋無關的字串分隔開，並且只取後面有關的字串
pre_data_num = 0
bs = 512
train_games, val_games = train_test_split(games, test_size=0.001)
val_x_list = []
val_y_list = []
val_x_pre_eight_list = []
for val_game_num, val_game in enumerate(val_games):
    val_move_list = val_game.split(',')

    for idx, val_move in enumerate(val_move_list):
        val_x = prepare_input(val_move_list[:idx])
        val_x_list.append(val_x)
        val_y = prepare_label(val_move_list[idx])
        val_y = torch.nn.functional.one_hot(val_y, 19*19)
        val_y = torch.tensor(val_y, dtype=torch.float32)
        val_y_list.append(val_y)

        if((idx+1) <= pre_data_num):
            for i in range(idx):
                val_x = torch.cat((val_x, val_x_list[idx-1-i][5]), dim=0)
            for i in range(pre_data_num - idx):
                val_x = torch.cat((val_x, torch.zeros((1, 19, 19))), dim=0)
               
        elif((idx+1) > pre_data_num):
            for i in range(pre_data_num):
                val_x = torch.cat((val_x, val_x_list[idx-1-i][5]), dim=0)
            
        val_x_pre_eight_list.append(val_x)


val_x_pre_eight_stack = torch.stack(val_x_pre_eight_list)
val_y_stack = torch.stack(val_y_list)

val_x_pre_eight_stack = val_x_pre_eight_stack.to(device)
val_y_stack = val_y_stack.to(device)

val_dataset = CustomDataset(val_x_pre_eight_stack, val_y_stack)
data_loader_val = DataLoader(dataset=val_dataset, batch_size=bs, shuffle=False)
model = torch.load('./kyu_models/model2300.pth')
model.eval()
accuracy_list_top1 = []
accuracy_list_top5 = []
with torch.no_grad():  # Disable gradient calculation during validation
    for val_x, val_y in tqdm(data_loader_val):
        predictions = model(val_x)
        top_k_values, top_k_indices = torch.topk(predictions, k=5, dim=1)
        val_y_values, val_y_indices = torch.topk(val_y, k=1)
        correct_predictions_top1 = torch.eq(top_k_indices[:, 0], val_y_indices.squeeze())
        correct_predictions_top5 = torch.any(torch.eq(top_k_indices, val_y_indices), dim=1)

        accuracy_top1 = torch.sum(correct_predictions_top1  == True) / len(val_y)
        accuracy_top5 = torch.sum(correct_predictions_top5  == True) / len(val_y)
        accuracy_list_top1.append(accuracy_top1.item())
        accuracy_list_top5.append(accuracy_top5.item())
        
        # print(f'Accuracy_top1: {accuracy_top1 * 100:.2f}%')
        # print(f'Accuracy_top5: {accuracy_top5 * 100:.2f}%')
print(f'top 1 avg acc:{sum(accuracy_list_top1)/len(accuracy_list_top1)}')
print(f'top 5 avg acc:{sum(accuracy_list_top5)/len(accuracy_list_top5)}')


## 評估playstyle之驗證集準確率

In [ ]:
df = open('./29_Training Dataset/Training Dataset/play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]
y_list = torch.tensor(game_styles)-1
y_list = torch.nn.functional.one_hot(y_list, 3)
y_list = torch.tensor(y_list, dtype = torch.float32)
bs = 64
pre_data_num = 7
train_games, val_games, train_y, val_y = train_test_split(games, y_list, test_size=0.004,random_state=64)
val_x_list = []
val_y_list = []
x = []


for val_game_num, val_game in enumerate(val_games):
    val_move_list = val_game.split(',')
    val_x = prepare_input(val_move_list)
    last_idx = len(val_move_list)-1
    if(last_idx < pre_data_num):
        for i in range(last_idx):
            val_x = torch.cat((val_x, prepare_input(val_move_list[:last_idx-1-i])[:2]), dim=0)
        for i in range(pre_data_num - last_idx):
            val_x = torch.cat((val_x, torch.zeros((2, 19, 19))), dim=0)      
    else:
        for i in range(pre_data_num):
            val_x = torch.cat((val_x, prepare_input(val_move_list[:last_idx-1-i])[:2]), dim=0)
    
    val_x_list.append(val_x)

val_x_list = torch.stack(val_x_list)

# val_y_stack = torch.stack(val_y_list)
val_x_list = val_x_list.to(device)
val_y = val_y.to(device)

val_dataset = CustomDataset(val_x_list, val_y)
data_loader_val = DataLoader(dataset=val_dataset, batch_size=len(val_games), shuffle=False)
model = torch.load('./playstyle_models/best_PS_model.pth')
model.eval()
            
total_val_loss = 0

loss_fn = torch.nn.CrossEntropyLoss()

with torch.no_grad(): 
    for val_x, val_y in tqdm(data_loader_val):
        val_outputs = model(val_x)
        val_loss = loss_fn(val_outputs, val_y)
        total_val_loss += val_loss.item() * val_x.shape[0]
        predicted_labels = torch.argmax(val_outputs, dim=1) 
        true_labels = torch.argmax(val_y, dim=1)

avg_loss_val = total_val_loss/len(val_dataset)

accuracy = torch.sum(predicted_labels == true_labels).item() / len(true_labels)


print(f'Validation Acc: {accuracy}')
print(f'Validation Loss: {avg_loss_val}')